# Figures

This notebook creates the following figures:

* visual field coverage plots
* polar angle histograms
* eccentricity distributions
* eccentricity-size relations
* timeseries plot

In [3]:
# imports
from __future__ import division
import os
import nibabel as nb
import numpy as np
import scipy as sp
import h5py
import matplotlib.pyplot as pl
import seaborn as sn
sn.set_style('ticks')
import glob
import statsmodels.api as sm
import pandas as pd
import shutil

import matplotlib as mpl
%matplotlib inline
mpl.rc_file_defaults()

### define directories:

In [58]:
# set this to your data directory:
data_dir = '/home/shared/2018/visual/hcp_cerebellum/'
repo_dir = '/home/vanes/git/hcp_cerebellum/'

### subdirectories

In [59]:
resource_dir = os.path.join(repo_dir,'resources')
retmap_dir = os.path.join(resource_dir,'volume_masks')

# setup figure directory
fig_dir = os.path.join(data_dir,'figs')
if not os.path.isdir(fig_dir): os.mkdir(fig_dir)

### settings

In [60]:
# these are the dimensions of the niftis
dims = {
    'ang':0,
    'ecc':1,
    'gain':2,
    'meanvol':3,
    'r2':4,
    'rfsize':5
}   

stim_radius = 8
avg_masktype = 'r2_spill_fix'
ind_masktype = 'r2_roi'

### define rois

In [61]:
# these refer to the indices in cerebellum_retmaps.nii
mask_names ={
'left_mOMV':7,
'right_mOMV':9,
'left_lOMV':8,
'right_lOMV':10,
    
'left_VIIb':5,
'right_VIIb':6,
    
'left_mVIIIb':3,
'right_mVIIIb':2,
'left_lVIIIb':4,
'right_lVIIIb':1,
}

# this is how the lateral/medial and left right hemisphere indices combine:
roi_combs = {
    'left_OMV':[7,8],
    'right_OMV':[9,10],
    'left_VIIb':[5],
    'right_VIIb':[6],
    'left_VIIIb':[3,4],
    'right_VIIIb':[1,2],

    'OMV':[7,8,9,10],
    'lOMV':[8,10],
    'mOMV':[7,9],
    'VIIb':[5,6],
    'VIIIb':[1,2,3,4],
    'lVIIIb':[1,4],
    'mVIIIb':[2,3],
}

roi_order = ['left_mOMV','right_mOMV','left_lOMV','right_lOMV','left_VIIb','right_VIIb','left_mVIIIb','right_mVIIIb','left_lVIIIb','right_lVIIIb']
roi_comb_order = ['mOMV','lOMV','VIIb','mVIIIb','lVIIIb']           

#### load data

In [62]:
# get the data
avgdata = nb.load(os.path.join(base_dir,'masked_niftis','r2_spill_fix','prfresults_subject_rank_avg.nii.gz')).get_data()
cer_retmaps = nb.load(os.path.join(retmap_dir,'cerebellum_retmaps.nii')).get_data()

#### function for determining roi color

In [8]:
def get_roi_color(mask):
    
    # determine base roi colors
    if mask in ['left_OMV','right_OMV','OMV']:
        c = '#6590CB'
    elif mask in ['left_VIIIb','right_VIIIb','VIIIb']:
        c = '#E55D5C'
    elif ('VIIb' in mask) + (mask=='VIIb'):
        c = '#E79F2A'
        
    # roi colors for lateral / medial separation:
    elif mask in ['left_lOMV','right_lOMV','lOMV']:
        c = '#6590CB'
    elif mask in ['left_mOMV','right_mOMV','mOMV']:
        c = '#0D5B99'
    elif mask in ['left_mVIIIb','right_mVIIIb','mVIIIb']:
        c = '#C03142'
    elif mask in ['left_lVIIIb','right_lVIIIb','lVIIIb']:
        c = '#E57F80'
        
    return c

### visualize pRF center + size distribution

In [22]:
f = pl.figure(figsize=(2,4.5))
for mi,mask in enumerate(roi_order):
    
    s = f.add_subplot(5,2,mi+1,aspect='equal')

    # determine this roi mask
    roimask = (cer_retmaps==(mask_names[mask]))

    # convert polar to cartesian:
    xs = np.cos(np.radians(np.ravel(avgdata[roimask,dims['ang']]))) * np.ravel(avgdata[roimask,dims['ecc']])
    ys = np.sin(np.radians(np.ravel(avgdata[roimask,dims['ang']]))) * np.ravel(avgdata[roimask,dims['ecc']])

    # get sizes
    sizes = np.ravel(avgdata[roimask,dims['rfsize']])

    # determine roi color
    c = get_roi_color(mask)

    # now draw a circle at rf size
    for x,y,sigma in zip(xs,ys,sizes):
        s.add_artist(pl.Circle((x,y),sigma, color=c,fill=False,alpha=0.25))

    # draw crosshair
    pl.axhline(0,lw=0.5,color='k')
    pl.axvline(0,lw=0.5,color='k')

    # draw dot at center 
    pl.plot(xs,ys,'o',color=c,ms=3,mec='w',mew=1,alpha=1)
    pl.xlim(-stim_radius,stim_radius)
    pl.ylim(-stim_radius,stim_radius)   

    # only draw the axis labels for VIIIb:
    if mask =='left_lVIIIb':
        pl.xticks([-stim_radius,0,stim_radius],['-%d'%stim_radius,0,'%.d'%stim_radius])
        pl.yticks([-stim_radius,0,stim_radius],['-%d'%stim_radius,0,'%.d'%stim_radius])
        sn.despine(offset=2)
    elif mask =='right_lVIIIb':
        pl.xticks([-stim_radius,0,stim_radius],['-%.d'%stim_radius,0,'%.d'%stim_radius])
        pl.yticks([])
        sn.despine(offset=2,left=True)
    else:
        pl.axis('off')
        pl.xticks([])
        pl.yticks([])                

pl.tight_layout()
f.savefig(os.path.join(fig_dir,'prf_scatter_avg_subject.pdf'))

### create eccentricity-size plot

In [45]:
# eccen-size plot
this_roi_comb_order = ['mVIIIb','lVIIIb','VIIb','mOMV','lOMV']

f = pl.figure(figsize=(1.25,1.25))
s = f.add_subplot(111)

all_max_eccs = []
all_max_sizes=[]
for mi,mask in enumerate(this_roi_comb_order):

    # determine this roi mask
    roimask = np.zeros_like(cer_retmaps).astype(bool)
    for subroi in roi_combs[mask]:
        roimask[(cer_retmaps==subroi)] = True

    # determine roi color
    c = get_roi_color(mask)  

    sizes = np.ravel(avgdata[roimask,dims['rfsize']])
    eccs = np.ravel(avgdata[roimask,dims['ecc']])

    maxecc = np.min([np.nanmax(eccs),stim_radius])

    v = (eccs < maxecc)

    # do linear regression with statsmodels
    y = sizes[v]
    X = eccs[v]
    X = sm.add_constant(X)     

    mod = sm.OLS(y, X)
    res = mod.fit()
    ci_intercept,ci_slope = res.conf_int(0.05)
    mean_intercept,mean_slope = res.params

    fit = np.polyval([mean_slope,mean_intercept],np.linspace(0,maxecc,15))
    fit_high = np.polyval([ci_slope[0],ci_intercept[1]],np.linspace(0,maxecc,15))
    fit_low = np.polyval([ci_slope[1],ci_intercept[0]],np.linspace(0,maxecc,15))

    pl.plot(np.linspace(0,maxecc,15),fit,color=c,lw=2)
    pl.fill_between(np.linspace(0,maxecc,15),fit_low,fit_high,alpha=0.25,color=c)

pl.xlim(0,stim_radius)  
pl.ylim(0,10)
pl.xticks([0,stim_radius])
pl.yticks([0,10])

sn.despine(offset=2)

pl.tight_layout()
f.savefig(os.path.join(fig_dir,'prf_eccsize_avg.pdf'))

### create eccentricity histogram

In [50]:
f = pl.figure(figsize=(1.25,2))
s = f.add_subplot(111)

dc = pd.DataFrame()
colors = []

for mi,mask in enumerate(roi_comb_order):

    # determine this roi mask
    roimask = np.zeros_like(cer_retmaps).astype(bool)
    for subroi in roi_combs[mask]:
        roimask[(cer_retmaps==subroi)] = True

    # get data
    eccs = np.ravel(avgdata[roimask,dims['ecc']])

    # get roi color and append to all
    c = get_roi_color(mask)
    colors.append(c)
    
    # get eccentricities, append with nans (otherwise df is shortened to roi with least voxels)
    dc[mask] = np.hstack([eccs,np.ones(100000-len(eccs))*np.nan])

sn.violinplot(data=dc,palette=colors,scale='width',orient='h',linewidth=0)

pl.xlim(0,stim_radius)
pl.xticks([0,stim_radius])
sn.despine(offset=2)
pl.xlabel('eccen (dva)')

pl.tight_layout(pad=0)
f.savefig(os.path.join(fig_dir,'ecc_histograms_avg.pdf'))

### polar histogram

In [65]:
for hemi in ['left','right']:
    f = pl.figure(figsize=(0.75,4.5))

    for mi,roi in enumerate(['mOMV','lOMV','VIIb','mVIIIb','lVIIIb']):
        mask = hemi+'_'+roi
        s = f.add_subplot(5,1,mi+1,projection='polar')

        # determine this roi mask
        roimask = (cer_retmaps==(mask_names[mask]))

        # determine color for this ROI
        c = get_roi_color(mask)

        # get data (convert to radians for polar plot)
        angles = np.radians(np.ravel(avgdata[roimask,dims['ang']]))

        # compute bins (let's do for 12 bins)
        bins_number = 12
        bins = np.linspace(0.0, 2 * np.pi, bins_number + 1)
        n, _, _ = pl.hist(angles, bins)
        width = (2 * np.pi) / bins_number
        
        # plot polar bars
        s.cla()        
        pl.bar(bins[:bins_number], n, width=width, bottom=0.0,ec=c,color=c)

        # plot settings
        pl.xticks([])
        pl.yticks([])
        pl.ylim(0,np.max(n))

    pl.tight_layout()
    f.savefig(os.path.join(fig_dir,'polar_histograms_avg_%s.pdf'%(hemi)))

### individual subject eccen-size relations

In [64]:
# eccen-size plot
for sgi,sjg in zip([range(5),range(88,93),range(176,181)],['top','mid','bottom']):
    
    f = pl.figure(figsize=(0.75*5,0.75))
    for mi,mask in enumerate(roi_comb_order):
        s = f.add_subplot(1,len(roi_comb_order),mi+1)

        # determine this roi mask
        roimask = np.zeros_like(cer_retmaps).astype(bool)
        for subroi in roi_combs[mask]:
            roimask[(cer_retmaps==subroi)] = True

        # determine ecc limit and roi color
        c = get_roi_color(mask)  

        for sj in sgi:
            
            this_sj_data = nb.load(os.path.join(data_dir,'masked_niftis','r2_roi','prfresults_subject_rank_%d.nii.gz'%sj)).get_data()

            try:                
                sizes = np.ravel(this_sj_data[roimask,dims['rfsize']])
                eccs = np.ravel(this_sj_data[roimask,dims['ecc']])                
                
                # determine valid voxels
                maxecc = np.min([np.nanmax(eccs),stim_radius])
                v = (eccs < maxecc)

                # do linear regression with statsmodels
                y = sizes[v]
                X = eccs[v]
                X = sm.add_constant(X)     


                mod = sm.OLS(y, X)
                res = mod.fit()
                ci_intercept,ci_slope = res.conf_int(0.05)
                mean_intercept,mean_slope = res.params

                fit = np.polyval([mean_slope,mean_intercept],np.linspace(0,maxecc,15))
                fit_high = np.polyval([ci_slope[0],ci_intercept[1]],np.linspace(0,maxecc,15))
                fit_low = np.polyval([ci_slope[1],ci_intercept[0]],np.linspace(0,maxecc,15))

                pl.plot(np.linspace(0,maxecc,15),fit,color=c,lw=1)
                pl.fill_between(np.linspace(0,maxecc,15),fit_low,fit_high,alpha=0.25,color=c)
                
            except:
                print 'could not fit %s ecc-size relation for %s subject %d (too little data)'%(mask,sjg,sj)


        pl.xlim(0,stim_radius)  
        pl.ylim(0,10)
        pl.xticks([0,stim_radius])
        pl.yticks([0,10])

        sn.despine(offset=2)

    pl.tight_layout()
    f.savefig(os.path.join(fig_dir,'prf_eccsize_all_%s.pdf'%(sjg)))

could not fit mOMV ecc-size relation for mid subject 89 (too little data)
could not fit lVIIIb ecc-size relation for mid subject 88 (too little data)
could not fit lVIIIb ecc-size relation for mid subject 90 (too little data)
could not fit lVIIIb ecc-size relation for mid subject 91 (too little data)
could not fit mOMV ecc-size relation for bottom subject 176 (too little data)
could not fit mOMV ecc-size relation for bottom subject 177 (too little data)
could not fit mOMV ecc-size relation for bottom subject 178 (too little data)
could not fit mOMV ecc-size relation for bottom subject 179 (too little data)
could not fit mOMV ecc-size relation for bottom subject 180 (too little data)
could not fit lOMV ecc-size relation for bottom subject 176 (too little data)
could not fit lOMV ecc-size relation for bottom subject 177 (too little data)
could not fit lOMV ecc-size relation for bottom subject 178 (too little data)
could not fit lOMV ecc-size relation for bottom subject 179 (too little da

# Timeseries plot

In this plot, we'll get the design matrix from the osf website, and use the pRF model that was fit on all data (wedges / rings / bars) to create a predicted fit for the average of the two bar runs (downloaded in notebook 1_export_ciftis).

In [9]:
# Popeye imports
import popeye.utilities as utils
from popeye.visual_stimulus import VisualStimulus
import popeye.css as css
import scipy as sp
import urllib
import zipfile
import copy

In [10]:
ap_dir = os.path.join(base_dir,'apertures.zip')
ap_url = 'https://osf.io/5sj3m/download'
up_ap_dir = os.path.join(base_dir,'apertures')
mat_dir = os.path.join(base_dir,'RETBARsmall.mat')

### download the design matrices

In [11]:
if not os.path.isfile(ap_dir):
    print('Downloading retinotopy design matrices...')
    urllib.urlretrieve(ap_url, ap_dir)  
    print('downloading done!')
else:
    print('dms already downloaded')

downloading done!


### unpacking the dms

In [12]:
if not os.path.isdir(up_ap_dir):
    print('unpacking dms')
    zip_ref = zipfile.ZipFile(ap_dir, 'r')
    zip_ref.extractall(ap_dir[:-4])
    zip_ref.close()
    # remove zip file
    os.remove(ap_dir)
else:
    print('dms already unpacked')

dms already unpacked


### move and remove files

In [13]:
if not os.path.isfile(mat_dir):
    os.rename(os.path.join(up_ap_dir,'apertures','RETBARsmall.mat'),mat_dir)
    shutil.rmtree(up_ap_dir)
else:
    print('dm ret file already moved')

dm ret file already moved


### now load in file

In [14]:
with h5py.File(mat_dir, 'r') as mat:
    visual_dm = mat['stim'].value.T

### convert file to popey visual stimulus

In [15]:
# Create stimulus design and define model
# ---------------------------------------
stimulus = VisualStimulus(  stim_arr = visual_dm,
                            viewing_distance = 102, 
                            screen_width = 29,
                            scale_factor = 1,
                            tr_length = 1.0,
                            dtype = np.short)

model_func = css.CompressiveSpatialSummationModel(  stimulus = stimulus,
                                                    hrf_model = utils.spm_hrf)
model_func.hrf_delay = 0



### load in the averaged timecourse data

In [40]:
# Get timeseries data
# ---------------------------------------
timeseries_fn = os.path.join(base_dir,'timeseries','avg_prf_timeseries.nii.gz')
timeseries_data = nb.load(timeseries_fn).get_data()

In [41]:
# zscore timeseries
avg = np.nanmean(timeseries_data,axis=-1)[:,:,:,np.newaxis]
std = np.nanstd(timeseries_data,axis=-1)[:,:,:,np.newaxis]
timeseries_data = (timeseries_data-avg)/std

### setup function that creates prediction and rescales

In [26]:
def create_prediction(voxel_idx,roimask):

    # get prf parameters
    ang = np.radians(avgdata[roimask,dims['ang']][voxel_idx])
    ecc = avgdata[roimask,dims['ecc']][voxel_idx]
    size = avgdata[roimask,dims['rfsize']][voxel_idx]

    # get ts data
    these_ts = timeseries_data[roimask]
    
    # convert to cartesian
    x = ecc * np.cos(ang)
    y = ecc * np.sin(ang)

    # this is the n used in the paper
    n = 0.05

    # calculate prediction
    pred = model_func.generate_prediction(x,y,size,n,1,0)
    
    # refit baseline and amp parameters (since fitted on different data)
    dm = np.mat(np.vstack([np.ones_like(pred),pred])).T
    t = these_ts[voxel_idx]
    intercept,slope = np.array(np.linalg.pinv(dm.T * dm) * dm.T * np.mat(t[:,np.newaxis]))     

    # scale prediction
    p =(intercept[0]+pred*slope[0])

    # compute new r2
    r2 = (sp.stats.pearsonr(p,t)[0]**2)*100
    
    return p,r2



### Determine best voxels for these data

In [35]:
these_r2s = {}
ecc_bands = [[0,2],[2,4],[4,6],[6,8]]
for ecc_band in ecc_bands:
    these_r2s[str(ecc_band)] = {}
    
    # select data from this ecc band
    eccs = avgdata[:,:,:,dims['ecc']]
    valid_eccs = (eccs>ecc_band[0])*(eccs<ecc_band[1])
    
    for mi,mask in enumerate(roi_comb_order):
        print('creating predictions for ecc_band %s, mask %s'%(ecc_band,mask))
        # determine this roi mask
        roimask = np.zeros_like(cer_retmaps).astype(bool)
        for subroi in roi_combs[mask]:
            roimask[(cer_retmaps==subroi)] = True

        roimask *= valid_eccs

        # apply mask to timeseries
        these_ts = timeseries_data[roimask]
    
        # now create a prediction for every voxel
        # on the averaged data and save the r2s
        r2s = []
        for v in range(roimask.sum()):

            p,r2 = create_prediction(v,roimask)

            # recompute r2 on this prediction and data
            r2s.append(r2)
            
        these_r2s[str(ecc_band)][mask] = r2s

creating predictions for ecc_band [0, 2], mask mOMV, 0%done
creating predictions for ecc_band [0, 2], mask lOMV, 0%done
creating predictions for ecc_band [0, 2], mask lOMV, 20%done
creating predictions for ecc_band [0, 2], mask lOMV, 40%done
creating predictions for ecc_band [0, 2], mask lOMV, 60%done
creating predictions for ecc_band [0, 2], mask lOMV, 80%done
creating predictions for ecc_band [0, 2], mask VIIb, 0%done
creating predictions for ecc_band [0, 2], mask mVIIIb, 0%done
creating predictions for ecc_band [0, 2], mask mVIIIb, 20%done
creating predictions for ecc_band [0, 2], mask mVIIIb, 40%done
creating predictions for ecc_band [0, 2], mask mVIIIb, 60%done
creating predictions for ecc_band [0, 2], mask mVIIIb, 80%done
creating predictions for ecc_band [0, 2], mask lVIIIb, 0%done
creating predictions for ecc_band [2, 4], mask VIIb, 0%done
creating predictions for ecc_band [2, 4], mask mVIIIb, 0%done
creating predictions for ecc_band [2, 4], mask lVIIIb, 0%done
creating predict

#### now we can create plot for the best voxels

In [65]:
# create plot
# ---------------------------------------
n_vox = 5
for ecc_band in ecc_bands:
    
    # select data from this ecc band
    eccs = avgdata[:,:,:,dims['ecc']]
    valid_eccs = (eccs>ecc_band[0])*(eccs<ecc_band[1])   
    
    for mi,mask in enumerate(roi_comb_order):

        # determine this roi mask
        roimask = np.zeros_like(cer_retmaps).astype(bool)
        for subroi in roi_combs[mask]:
            roimask[(cer_retmaps==subroi)] = True

        roimask *= valid_eccs

        # apply mask to timeseries
        these_ts = timeseries_data[roimask]

        # now determine best voxel based on fits to this data
        r2s = these_r2s[str(ecc_band)][mask]
        best_voxels = np.argsort(r2s)[::-1][:n_vox]

        # and create plots for these voxels
        for best_voxel in best_voxels:

            # get this timeseries
            timeseries = these_ts[best_voxel]

            pred,r2 = create_prediction(best_voxel,roimask)
            
            # generate pRF
            res = 501

            # get pRF parameters for visualization
            ang = np.radians(avgdata[roimask,dims['ang']][best_voxel])
            ecc = avgdata[roimask,dims['ecc']][best_voxel]
            size = avgdata[roimask,dims['rfsize']][best_voxel]
            
            x = ecc * np.cos(ang)
            y = ecc * np.sin(ang)

            f=pl.figure(figsize=(4,1.25))            
            
            # first plot the prf parameters
            s = f.add_subplot(141)
            pl.text(0,0,'ecc: %.2f\nangle: %.2f\nsize: %.2f\nR2: %.2f'%(ecc,ang,size,r2),
                    horizontalalignment='right',verticalalignment='center')            
            pl.xlim(-5,5)
            pl.ylim(-5,5)
            pl.axis('off')
            
        
            # now create a pRF visualization plot
            s = f.add_subplot(142,aspect='equal')

            c = get_roi_color(mask)

            # crosshair
            pl.axhline(0,lw=0.5,color='k')
            pl.axvline(0,lw=0.5,color='k')
            
            # for prf size
            s.add_artist(pl.Circle((x,y),size, color='r',fill=False,alpha=0.25))
            # prf center
            pl.plot(x,y,'o',color='r',ms=3,mec='w',mew=1,alpha=1)

            # plot properties
            pl.xlim(-stim_radius,stim_radius)
            pl.ylim(-stim_radius,stim_radius) 
            pl.xticks([-stim_radius,0,stim_radius])
            pl.yticks([-stim_radius,0,stim_radius])
            pl.xlabel('visual field x (dva)')
            pl.ylabel('visual field y (dva)')
            sn.despine(offset=2)
            
            # and finally the pRF timeseries prediction
            s = f.add_subplot(122)
    
            # timeseries
            pl.plot(timeseries,'o--',color='k',ms=1.5)
            # prediction
            pl.plot(pred,color='r',lw=1.5)

            # this is the gray/white shading to indicate stimulus design
            ylims=s.get_ylim()
            pl.fill_between([0,16],[ylims[0],ylims[0]],[ylims[1],ylims[1]],color='w')
            pl.fill_between([16,48],[ylims[0],ylims[0]],[ylims[1],ylims[1]],color='k',alpha=0.1)
            pl.fill_between([48,80],[ylims[0],ylims[0]],[ylims[1],ylims[1]],color='w')
            pl.fill_between([80,112],[ylims[0],ylims[0]],[ylims[1],ylims[1]],color='k',alpha=0.1)
            pl.fill_between([112,144],[ylims[0],ylims[0]],[ylims[1],ylims[1]],color='w')
            pl.fill_between([144,156],[ylims[0],ylims[0]],[ylims[1],ylims[1]],color='k',alpha=0.1)
            pl.fill_between([156,188],[ylims[0],ylims[0]],[ylims[1],ylims[1]],color='w')
            pl.fill_between([188,220],[ylims[0],ylims[0]],[ylims[1],ylims[1]],color='k',alpha=0.1)
            pl.fill_between([220,252],[ylims[0],ylims[0]],[ylims[1],ylims[1]],color='w')
            pl.fill_between([252,284],[ylims[0],ylims[0]],[ylims[1],ylims[1]],color='k',alpha=0.1) 
            pl.fill_between([284,300],[ylims[0],ylims[0]],[ylims[1],ylims[1]],color='w')             
                
            # plot properties
            sn.despine(offset=2)
            pl.xlabel('time (s)')
            pl.ylabel('BOLD (z-score)')

            # save figure
            pl.tight_layout()
            f.savefig(os.path.join(fig_dir,'pred_%s_eccband_%s_v_%d.pdf'%(mask,ecc_band,best_voxel)))
            